# KAIROS-ARK: The Operating System for Agentic AI

Welcome to the official demo notebook for **KAIROS-ARK**.

This notebook demonstrates the core capabilities of the KAIROS kernel:
1.  **Deterministic Execution** (Hello World)
2.  **True Parallelism** (Multi-threaded implementation)
3.  **Security Policy Engine** (Kernel-level restrictions)
4.  **Performance Benchmarks** (Throughput & Zero-Copy Memory)

> **Note:** This runs entirely natively in Rust. No Python logic is executed in the scheduler hot path.

## 1. Installation

First, we install the package from PyPI.

In [ ]:
!pip install kairos-ark

## 2. Hello World Agent

Let's create a simple deterministic agent. KAIROS-ARK guarantees that running this agent with the same seed will produce **bit-for-bit identical** audit logs, every single time.

In [ ]:
from kairos_ark import Agent
import json

# 1. Initialize Agent with a fixed seed for determinism
agent = Agent(seed=42)

# 2. Define simple tools (nodes)
# Note: Handlers in this demo are parameterless for simplicity
agent.add_node("fetch_data", lambda: {"data": "raw_input_from_sensor"})
agent.add_node("process_data", lambda: {"status": "processed", "value": "RAW_INPUT_FROM_SENSOR"})

# 3. Connect the workflow graph
agent.connect("fetch_data", "process_data")

# 4. Execute starting from 'fetch_data'
print("🚀 Executing Workflow...")
results = agent.execute("fetch_data")

print("\n✅ Execution Complete!")
print("Results:", json.dumps(results, indent=2))

## 3. True Parallel Execution

Standard Python `asyncio` loops are single-threaded (GIL-bound). KAIROS-ARK provides a multi-threaded executor to achieve **true parallel fan-out** for blocking tasks.

Here, we simulate 3 blocking tasks running simultaneously.

In [ ]:
import time

# Define a blocking task that sleeps
def heavy_task(name):
    print(f"[{name}] Starting...")
    time.sleep(1.0) # Sleep for 1 second
    print(f"[{name}] Done!")
    return f"{name}_result"

# Add parallel nodes
agent.add_node("task_a", lambda: heavy_task("A"))
agent.add_node("task_b", lambda: heavy_task("B"))
agent.add_node("task_c", lambda: heavy_task("C"))

print("\n⚡ Starting Parallel Execution (Expect ~1.0s total time, NOT 3.0s)...")
start_time = time.time()

# Use run_parallel helper to bypass the GIL for Python tasks
results = agent.run_parallel(["task_a", "task_b", "task_c"])

end_time = time.time()
print(f"\n⏱️ Total Time: {end_time - start_time:.4f} seconds")

## 4. Kernel-Level Security Policy

One of KAIROS-ARK's unique features is its **Policy Engine**. You can restrict what an agent can do (network access, file system) at the kernel level.

In [ ]:
from kairos_ark import Policy, Cap

# 1. Register a tool that *requires* Network Access
agent.register_tool(
    "sensitive_web_tool",
    lambda: "This should be blocked",
    required_capabilities=[Cap.NET_ACCESS]  # Requires Network
)

# 2. Define a Policy that *blocks* Network Access
policy = Policy(
    allowed_capabilities=[Cap.LLM_CALL],  # Only allow LLM, NO Network
    forbidden_content=["API_KEY"]
)

# 3. Enforce the Policy
agent.set_policy(policy)
print("\n🛡️  Policy enforced: Network Access is BLOCKED")

# 4. Check if the tool is allowed
allowed, reason = agent.check_tool_capability("sensitive_web_tool")

if not allowed:
    print(f"✅ Security System Working: {reason}")
else:
    print("❌ Security Check Failed: Tool was allowed (Unexpected)")

## 5. Performance Benchmarks

Let's put the Rust kernel to the test.

### 5.1 High-Throughput Test
We will create a chain of 1,000 nodes and execute them to measure the raw overhead of the scheduler.

In [ ]:
# Reset agent
agent.clear()
agent = Agent(seed=123)

count = 1000
print(f"Creating {count} nodes...")
start_setup = time.time()

# Build linear chain
for i in range(count):
    agent.add_node(f"node_{i}", lambda: "ok")
    if i > 0:
        agent.connect(f"node_{i-1}", f"node_{i}")

setup_time = time.time() - start_setup
print(f"Setup Time: {setup_time:.4f}s")

print(f"Executing {count} nodes chain...")
start_exec = time.time()

agent.execute("node_0")

exec_time = time.time() - start_exec

print(f"Execution Time: {exec_time:.4f}s")
print(f"Throughput: {count / exec_time:.2f} nodes/sec")

### 5.2 Zero-Copy Shared Memory
KAIROS-ARK has a built-in shared memory pool for passing large objects (like images or embeddings) between nodes without serialization overhead. 

Here we check writing 50MB of data.

In [ ]:
import os

# Create 50MB of dummy data (Default shared memory limit is 64MB)
data_size = 50 * 1024 * 1024 # 50 MB
print(f"Allocating {data_size / 1024 / 1024} MB of data...")
large_data = b"x" * data_size

start_write = time.time()

# Write directly to kernel shared memory (returns a lightweight handle ID)
handle_id = agent.kernel.write_shared(large_data)

write_time = time.time() - start_write
print(f"Write Time: {write_time:.6f}s (Handle ID: {handle_id})")

# Define a node that receives only the handle ID, not the data payload
def process_large_data():
    # In a real app, this node could pass the handle to another process or 
    # read only a slice of the data without copying everything.
    return {"status": "read_complete", "handle": handle_id}

agent.add_node("process_large", process_large_data)
res = agent.execute("process_large")

print("Read Result:", res[0]["output"])